# Team Challenge SQL

## Parte 1: SQL Murder

### Enunciado

<div align="center">
<blockquote>
<p><em>A crime has taken place and the detective needs your help.</em></p>
<p><em>The detective gave you the crime scene report, but you somehow lost it.</em></p>
<p><em>You vaguely remember that the crime was a <strong>murder</strong> that occurred sometime on <strong>Jan.15, 2018</strong> and that it took place in <strong>SQL City</strong>.</em></p>
<p><em>Start by retrieving the corresponding crime scene report from the police department's database.</em></p>
</blockquote>
</div>

In [1]:
# Importamos librerías
import pandas as pd
import sqlite3

# Muestra todo el texto de las celdas del DataFrame
pd.set_option('display.max_colwidth', None)

In [2]:
# Conectamos con la base de datos 
connection = sqlite3.connect("./data/sql-murder-mystery.db")

# Creamos un cursor para acceder a la conexión
cursor = connection.cursor()

# Creamos una función para la ejecución de queries
def sql_query(query):
    return pd.read_sql(query, connection)

In [10]:
# Echamos un primer vistazo a la tabla con los informes de crímenes
query = '''
SELECT *
FROM crime_scene_report
LIMIT 10     
'''
sql_query(query)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his head...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City
5,20180115,theft,Big Bully stole my lunch money!,Chicago
6,20180115,fraud,"Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do\n eiusmod tempor incididunt ut labore et dolore magna aliqua.",Seattle
7,20170712,theft,"A lone hunter stalks the night, firing arrows into the Darkness.\n There is no hiding, no escape. In the distance, the beast\n falters, tethered to the void. The killing blow comes without\n hesitation, without mercy.",SQL City
8,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it is a thing of\n legend, both past and future.",SQL City
9,20171110,robbery,"The Gjallarhorn shoulder-mounted rocket system was forged from\n the armor of Guardians who fell at the Twilight Gap. Gifted\n to the survivors of that terrible battle, the Gjallarhorn\n is seen as a symbol of honor and survival.",SQL City


1.Primero, buscamos el informe con el tipo ‘asesinato’, filtrado a SQL City y con fecha 15 de Enero, 2018:

In [11]:
query = '''
SELECT *
FROM crime_scene_report
WHERE type = "murder" AND city = "SQL City" AND date = 20180115
'''
sql_query(query)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [ ]:
# Miramos qué pinta tiene la tabla 'person'
query = '''
SELECT *
FROM person
LIMIT 5
'''
sql_query(query)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684


Para buscar al primer testigo, como vive en la última casa de la calle usamos MAX()

In [12]:
query = '''
SELECT name, address_street_name, MAX(address_number), id
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC
'''
sql_query(query)

,name,address_street_name,MAX(address_number),id
0,Morty Schapiro,Northwestern Dr,4919,14887


La segunda testigo se llama Annabel y  vive en la calle Franklin Ave

In [17]:
query = '''
SELECT name, address_street_name, id
FROM person
WHERE address_street_name = "Franklin Ave" AND  name LIKE "Annabel%"
'''
sql_query(query)

,name,address_street_name,id
0,Annabel Miller,Franklin Ave,16371


Ahora buscamos el report de ambos testigos uniendo la tabla 'interview' con la tabla 
'person':

In [18]:
query = '''
SELECT i.*, p.name
FROM interview AS i
INNER JOIN person AS p
ON i.person_id = p.id
WHERE i.person_id = 14887 OR p.id = 16371
'''
sql_query(query)

,person_id,transcript,name
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W"".",Morty Schapiro
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.",Annabel Miller


Y pasamos a comprobar las pistas de cada uno, primero la de Morty, incluyendo 
que es miembro del gimnasio, el número de gimnasio que empieza por 48Z, que 
es gold member y la matrícula del coche que contiene H42W:

In [ ]:
query = '''
SELECT *
FROM get_fit_now_member
LIMIT 5
'''
sql_query(query)

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,AOE21,39426,Noe Locascio,20171005,regular
2,2PN28,63823,Jeromy Heitschmidt,20180215,silver
3,0YJ24,80651,Waneta Wellard,20171206,gold
4,3A08L,32858,Mei Bianchin,20170401,silver


In [20]:
query = '''
SELECT *
FROM drivers_license
LIMIT 5
'''
sql_query(query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,100460,63,72,brown,brown,female,XF02T6,Cadillac,SRX
2,101029,62,74,green,green,female,VKY5KR,Scion,xB
3,101198,43,54,amber,brown,female,Y5NZ08,Nissan,Rogue
4,101255,18,79,blue,grey,female,5162Z1,Lexus,GS


Unimos 3 tablas: get_fit_now_member con person y drivers_license

In [ ]:
query = '''
SELECT g.id, g.membership_status, g.person_id, p.name, d.plate_number
FROM get_fit_now_member as g
INNER JOIN person AS p
ON g.person_id = p.id
INNER JOIN drivers_license AS d
ON p.license_id = d.id
WHERE g.membership_status = "gold"
    AND g.id LIKE "48Z%"
    AND d.plate_number LIKE "%H42W%"
'''
sql_query(query)

,id,membership_status,person_id,name,plate_number
0,48Z55,gold,67318,Jeremy Bowers,0H42W2


Hemos obtenido un nombre!! Ahora vamos a comprobar la pista de Annabel, uniendo las tablas de 
'get_fit_now_check_in' y 'get_fit_now_member':

In [26]:
query = '''
SELECT *
FROM get_fit_now_check_in
LIMIT 5
'''
sql_query(query)

,membership_id,check_in_date,check_in_time,check_out_time
0,NL318,20180212,329,365
1,NL318,20170811,469,920
2,NL318,20180429,506,554
3,NL318,20180128,124,759
4,NL318,20171027,418,1019


In [ ]:
query = '''
SELECT c.membership_id, c.check_in_date, g.name
FROM get_fit_now_check_in as c
INNER JOIN get_fit_now_member as g
ON c.membership_id = g.id
WHERE c.check_in_date = 20180109 AND g.id LIKE "48Z%"
'''
sql_query(query)

,membership_id,check_in_date,name
0,48Z7A,20180109,Joe Germuska
1,48Z55,20180109,Jeremy Bowers


Coincide un nombre en ambos: Jeremy Bowers con ID 67318, probamos el 
resultado: 

In [41]:
connection.execute("INSERT INTO solution VALUES (1, 'Jeremy Bowers')")

query = '''
SELECT value
FROM solution
'''
sql_query(query)

,value
0,"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


Obtenemos un nuevo prompt con pistas ya que hay otra persona detrás del asesinato, 
entonces buscamos su testimonio por ID:

In [42]:
query = '''
SELECT *
FROM interview
WHERE interview.person_id = 67318
'''
sql_query(query)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


Y nos da varias pistas: que tiene dinero (organizamos los valores de income en 
orden descendente para obtener primero el valor más alto), la altura, el color del 
cabello, el tipo de coche que tiene, la fiesta a la que asistió, la cantidad de 
veces y la fecha.

Unimos primero las tablas 'person' y 'drivers_license':

In [45]:
query = '''
SELECT p.id, p.name, d.height, d.hair_color, d.car_make, d.car_model
FROM drivers_license AS d
INNER JOIN person AS p
ON p.license_id = d.id
WHERE d.height BETWEEN "65" AND "67"
    AND d.hair_color = "red"
    AND d.car_make = "Tesla"
    AND d.car_model = "Model S"
'''
sql_query(query)

,id,name,height,hair_color,car_make,car_model
0,78881,Red Korb,65,red,Tesla,Model S
1,90700,Regina George,66,red,Tesla,Model S
2,99716,Miranda Priestly,66,red,Tesla,Model S


Obtenemos tres personas. Ahora unimos las tablas 'person', 'income' y 'facebook_event_checkin' sabiendo los ID's:

In [46]:
query = '''
SELECT p.id, p.name, f.event_name, f.date, i.annual_income
FROM facebook_event_checkin AS f
INNER JOIN person AS p
ON p.id = f.person_id
INNER JOIN income as i
ON p.ssn = i.ssn
WHERE p.id in (78881, 90700, 99716)
ORDER BY annual_income DESC
'''
sql_query(query)

,id,name,event_name,date,annual_income
0,99716,Miranda Priestly,SQL Symphony Concert,20171206,310000
1,99716,Miranda Priestly,SQL Symphony Concert,20171212,310000
2,99716,Miranda Priestly,SQL Symphony Concert,20171229,310000


Tenemos una pista sólida, comprobamos el resultado:

In [47]:
connection.execute("INSERT INTO solution VALUES (1, 'Miranda Priestly')")

query = '''
SELECT value
FROM solution
'''
sql_query(query)

,value
0,"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"
